<a href="https://colab.research.google.com/github/FrkAk/Deep_Learning_Workshop/blob/master/reading-letters/ReadingLetters.ipynb" target="_parent"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# How to create a basic brain for recognition of handwritten letters



In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow_datasets as tfds
import pandas as pd
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Data

We will use two different dataset to teach our machine to how read letters. First one is famous datasets MNIST for digit and another one is Kaggle AZ dataset from Kaggle.

MNIST dataset is already available in keras library or alternatively in tensorflow dataset library so do not worry about it. However, for AZ dataset we will manually download it from kaggle. (≈700MB)

### Load the MNIST dataset:
Make sure that tensorflow_datasets is already installed. If not simply type terminal: pip install tensorflow_datasets

### Load the AZ dataset:

Use the following link to download to dataset on to your machine and unzip to following directory ~/AZ_dataset/.
https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format


In [2]:

(mnist_train, mnist_test), mnist_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [3]:

az_dataset_directory = "AZ_dataset/A_ZHandwrittenData.csv"
az_data = pd.read_csv(az_dataset_directory,header= None)

AZ dataset contains labels and images at one row. First row is Label and rest is image itself.

In [4]:
az_label = az_data.values[:,0]
az_image = az_data.values[:,1:]
az_image = np.reshape(az_image,(az_image.shape[0],28,28,1))

Before go through create our model these two dataset needs to be merged. Since MNIST dataset is already done with library call, we try to adapt AZ dataset to MNIST.

To split our training and validation sets, we will use sklearn library.

But first we need to find size of test and train set to merge without headache.

In [5]:
from sklearn.model_selection import train_test_split

test_ratio = float(mnist_info.splits['test'].shard_lengths[0] / mnist_info.splits['train'].shard_lengths[0])
az_train_im, az_test_im, az_train_label, az_test_label = train_test_split(az_image,az_label,test_size=test_ratio)



In [6]:

mnist = keras.datasets.mnist
(train_images_mnist,train_labels_mnist),(test_images_mnist,test_labels_mnist) = mnist.load_data()
# images are reshaped to be used by the flow method of a keras ImageGenerator
train_images_mnist = np.reshape(train_images_mnist,(train_images_mnist.shape[0],28,28,1))
test_images_mnist = np.reshape(test_images_mnist,(test_images_mnist.shape[0],28,28,1))

In [7]:
train_labels_mnist = train_labels_mnist + max(az_label)+1
test_labels_mnist = test_labels_mnist + max(az_label)+1
# #shift mnist labels
# mnist_train.element_spec[1] = mnist_train.element_spec[1] + max(az_label)+1
# mnist_test.element_spec[1] = mnist_test.element_spec[1] + max(az_label)+1


In [8]:
# concatenate datasets
train_images = np.concatenate((az_train_im,train_images_mnist),axis=0)
train_labels = np.concatenate((az_train_label,train_labels_mnist))
test_images = np.concatenate((az_test_im,test_images_mnist),axis=0)
test_labels = np.concatenate((az_test_label,test_labels_mnist))

total_class = len(np.unique(train_labels))

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(8,(3,3),activation='relu'),
    #tf.keras.layers.Dropout(0,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(total_class, activation='softmax')
])

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 8)           1160      
                                                                 
 flatten (Flatten)           (None, 72)                0

In [11]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest'
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255
)


In [12]:
# Flow training images in batches using generator
train_generator = train_datagen.flow(train_images, train_labels, batch_size=50, shuffle=True)
validation_generator = test_datagen.flow(test_images, test_labels, batch_size=50, shuffle=True)

In [13]:
# Create a callback that saves the model's weights every 10 epochs
save_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='saved_model/',
    verbose=1,
    save_weights_only=True,
    save_freq= 5)
early_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs= 100,
    batch_size = 50,
    verbose=2,
    callbacks=[save_callback,early_callback]
)

Epoch 1/100
